In [1]:
import openpyxl
from openpyxl.styles import Border, Side
from datetime import datetime

wb = openpyxl.Workbook()
ws = wb.active

#　固定データ
ws['B2'].value = '請求書'
ws['B4'].value = '株式会社ABC'
ws['B5'].value = '〒101-0022 東京都千代田区神田練塀町300'
ws['B6'].value = 'TEL:03-1234-5678 FAX:03-1234-5678'
ws['B7'].value = '担当者名:鈴木一郎 様'
ws['F4'].value = 'No.'
ws['F5'].value = '日付'


#　変動データ

 #　現在の日付（年月日）を取得して、変数todayに代入する
today = datetime.today()
formatted_today = today.strftime('%Y/%m/%d')
 #　管理No
num = 1
formatted_num = f"{num:04}"

ws['G4'] = formatted_num
ws['G5'] = formatted_today

 #　表の固定ヘッダーを書き込む
header = ['商品名', '数量', '単価', '金額']
for i, header_name in enumerate(header, start=1):
    ws.cell(row=10, column=i+1, value=header_name)

 #　表の変動データを書き込む
data = [
    ['商品A', 2, 10000, 20000],
    ['商品B', 1, 15000, 15000],
]
for j, data_value in enumerate(data, start=1):
    for i, value in enumerate(data_value, start=1):
        ws.cell(row=10+j, column=i+1, value=value)

 #　11行目から変動が想定される最終行までの数値を合計
total = 0
for row in ws.iter_rows(min_row=11, max_row=ws.max_row, min_col=5, max_col=5):
    for cell in row:
        total = total + cell.value 
 #　合計数値を最終行の5列目に反映
ws.cell(row=ws.max_row+1, column=5, value=total)


# 罫線のスタイルを定義
thin_border = Side(border_style="thin")
border = Border(top=thin_border, left=thin_border, right=thin_border, bottom=thin_border)
# B10からEの最終行までのセルに対して罫線を設定。
for row in ws['B10':'E'+str(ws.max_row)]:
    for cell in row:
        cell.border = border


 #　消費税率
tax_rate = 0.1
 #　税金の計算
tax_value = total * tax_rate
 #　税込金額の計算
tax_included = total + tax_value

 #　合計、消費税、税込合計の数値を反映
data2 = [
    [None, '合計', None, None, total],
    [None, '消費税',None, None, tax_value],
    [None, '税込合計',None, None, tax_included]
]
  #　空行の追加
ws.append([])
  #　data2を反映
for row_data2 in data2:
    ws.append(row_data2)


# Eの最終行からBの最終から-2行までのセルに対して罫線を設定。
for row in ws['B'+str(ws.max_row-2):'E'+str(ws.max_row)]:
    for cell in row:
        cell.border = border


#　日付のtodayオブジェクトを、特定のフォーマットで文字列に変換する
filename = f"請求書_{today.strftime('%Y%m%d')}.xlsx"
wb.save(filename)